In [ ]:
# %pip install mimesis
# %pip install polars

In [2]:
import time 
import os
from mimesis import Person, Address, Numeric
import polars as pl
from concurrent.futures import ThreadPoolExecutor
import csv



In [3]:
#write out datasets
ROW_CNT: int = 10_000_000
BATCH_SIZE: int = 10000

def generate_datasets(batch_nbr: int) -> None:
    peep = Person()
    adrs = Address()
    num = Numeric()

    f_path = os.path.expanduser(f'~/test_dummy_data/polars2/data{batch_nbr}.csv')
    with open(f_path, 'w', newline='') as csvfile:
        fieldnames = ['first_name', 'last_name', 'address_txt', 'zip_cd', 'city', 'state', 'net_worth']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for _ in range(ROW_CNT // BATCH_SIZE):
            rows = []
            for _ in range(BATCH_SIZE):
                rows.append({
                    'first_name': peep.first_name(),
                    'last_name': peep.last_name(),
                    'address_txt': adrs.address(),
                    'zip_cd': adrs.zip_code(),
                    'city': adrs.city(),
                    'state': adrs.state(),
                    'net_worth': num.integer_number(0, 50000)
                })
            writer.writerows(rows)

In [4]:
def generate_datasets_parallel(num_batches: int, max_workers: int) -> None:
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(generate_datasets, range(num_batches))

In [5]:
start_time = time.time() 
generate_datasets_parallel(10, 5)
end_time = time.time()  # Record the end time
total_time = end_time - start_time  # Calculate the total time taken
print(f"Total time taken: {total_time} seconds")

In [13]:

df = pl.read_csv('~/test_dummy_data/polars/data*.csv')

"""
    Update: Used the go fd fake data generator to generate 100M rows; it did it in about 5 minutes
    -- add a link for that to this test harness
"""

## TODO: Aggregate data up, add timestamp column, write to parquet


10000000


In [31]:
df2 = df.group_by("zip_cd").agg(
     pl.col("net_worth").sum().name.suffix("_tot")
    ,pl.col("first_name").n_unique().name.suffix("_cnt")
    ,pl.col("city").count().alias("city_cnt")
)

df2.head(5)
df2.write_csv(file="~/test.csv",include_header=True, separator="|")